In [1]:
import sys
sys.executable = '/home/chao/anaconda3/envs/cantera38/lib/python3.8/site-packages'
python_path = ['/home/chao/cantera/build/python', '/home/chao/RMG-Py', '/home/chao', '/home/chao/anaconda3/envs/cantera38/lib/python38.zip', '/home/chao/anaconda3/envs/cantera38/lib/python3.8', '/home/chao/anaconda3/envs/cantera38/lib/python3.8/lib-dynload', '/home/chao/anaconda3/envs/cantera38/lib/python3.8/site-packages']
for path in python_path:
    sys.path.append(path)

In [2]:
# First, import cantera, with the nickname `ct` to save us some typing later.
import cantera as ct

# Then the usual suspects:
import numpy as np
%matplotlib inline
from matplotlib import pyplot as plt
from bokeh.plotting import figure, output_file, show

import cantera as ct
print('Running Cantera version: ' + ct.__version__)
ct.__file__ = '/home/chao/cantera/build/python/__init__.py'
print(ct.__file__)

Running Cantera version: 2.5.0a4
/home/chao/cantera/build/python/__init__.py


In [3]:
gas = ct.Solution('cvode_error.yaml')
gas()
len(gas.reactions())


  gas:

       temperature   300 K
          pressure   1.0132e+05 Pa
           density   1.138 kg/m^3
  mean mol. weight   28.014 kg/kmol
   phase of matter   gas

                          1 kg             1 kmol     
                     ---------------   ---------------
          enthalpy            2076.5             58170  J
   internal energy            -86963       -2.4362e+06  J
           entropy            6842.9         1.917e+05  J/K
    Gibbs function       -2.0508e+06       -5.7451e+07  J
 heat capacity c_p            1038.3             29086  J/K
 heat capacity c_v            741.47             20772  J/K

                      mass frac. Y      mole frac. X     chem. pot. / RT
                     ---------------   ---------------   ---------------
                N2                 1                 1           -23.032
     [ +707 minor]                 0                 0  



23060

In [4]:
def get_ignition_delay(temperature, pressure = 40.,
                       stoichiometry = 1.0, plot = False):
    """
    Get the ignition delay time in miliseconds, at the specified
    temperature (K), pressure (bar), and stoichiometry 
    (stoichiometric = 1.0, fuel-rich > 1.0, oxygen-rich < 1.0).
    Default pressure is 10.0 bar, default stoichoimetry is 1.0.
    If plot=True then it draws a plot (default is False).
    """
    oxygen_mole = 1. 
    nitrogen_mole = 4*oxygen_mole
    heptane_mole = stoichiometry/11
    X_string  = 'C7H16(1):0.009465215, O2(2):0.108849976, N2:0.4'
    #X_string = 'CHPD(1):0.00622, C7H16(417):0.00463, O2(2):0.10880, N2:0.88035'                      #need change if cti changes

    gas.TPX = temperature, pressure*1e5, X_string
    reactor = ct.IdealGasReactor(gas)
    reactor_network = ct.ReactorNet([reactor])

    time = 0.0
    end_time = 1e-1 
    
    # Use lists instead of arrays, so they can be any length
    times = []
    concentrations = []
    pressures = []
    temperatures = []
    
    print_data = True
    while time < end_time:
        time = reactor_network.time
        times.append(time)
        temperatures.append(reactor.T)
        pressures.append(reactor.thermo.P)
        concentrations.append(reactor.thermo.concentrations)
        # take a timestep
        # the size of the step will be determined by the ODE solver
        # depending on how quickly things are changing.
        reactor_network.step()
    print("Reached end time {0:.2f} ms in {1} steps".format(times[-1]*1e3, len(times)))
    # convert the lists into arrays
    concentrations = np.array(concentrations)
    times = np.array(times)
    pressures = np.array(pressures)
    temperatures = np.array(temperatures)
   
    if plot:
        plt.subplot(2,1,1)
        plt.plot(times*1e3, pressures/1e5)
        plt.ylabel("Pressure (bar)", color='b')
        ax2 = plt.gca().twinx()
        ax2.set_ylabel('Temperature (K)', color='r')
        ax2.plot(times*1e3, temperatures, 'r')
    i_ch = gas.species_index('CH(16)')                                                         #need change if cti change
    i_o2 = gas.species_index('O2(2)')                                                          #need change if cti change
    excited_oh_generation = concentrations[:,i_o2] * concentrations[:,i_ch]
    if plot:
        plt.subplot(2,1,2)
        plt.plot(times*1e3, excited_oh_generation, 'g')
        plt.ylabel("OH* emission")
        plt.ylim(0,max(1e-18,1.1*max(excited_oh_generation)))
        plt.xlabel("Time (ms)")
        plt.tight_layout()
        plt.show()
    step_with_highest_oh_gen = excited_oh_generation.argmax()
    if step_with_highest_oh_gen > 1 and excited_oh_generation.max()>1e-20:
        ignition_time_ms = 1e3 * times[step_with_highest_oh_gen]
        print("At {0} K {1} bar, ignition delay time is {2} ms".format(temperature, pressure, ignition_time_ms))
        return ignition_time_ms
    else:
        print("At {0} K {1} bar, no ignition detected".format(temperature, pressure))
        return np.infty
        

In [5]:
# before change the code 
get_ignition_delay(791.7, 40)

CanteraError: 
***********************************************************************
CanteraError thrown by CVodesIntegrator::step:
CVodes error encountered. Error code: -3

At t = 0.000223256 and h = 1.67865e-14, the error test failed repeatedly or with |h| = hmin.
Components with largest weighted error estimates:
660: 469.1331845281508 
657: -406.8485600265133 
7: 14.724462168490367 
664: 0.012463966082660307 
2: -0.006975787101004809 
661: -0.0012161505126923986 
14: 0.0008966877668858371 
665: 0.0005651965912846613 
663: 0.0005304535754452601 
76: 5.3872026829489106e-05 
***********************************************************************


In [5]:
# after change the code 
get_ignition_delay(791.7, 40)

CanteraError: 
***********************************************************************
CanteraError thrown by ReactorNet::componentName:
Index out of bounds
***********************************************************************


To prove the indices are not out of bound, I use the component_name function to return the component names of the indices shown in the CVODE error message

In [6]:
def exception():
    try:
        get_ignition_delay(791.7, 40)
    except Exception as e:
        return str(e)
e = exception()
oxygen_mole = 1. 
nitrogen_mole = 4*oxygen_mole
heptane_mole = 1/11
X_string  = 'C7H16(1):0.009465215, O2(2):0.108849976, N2:0.4'
#X_string = 'CHPD(1):0.00622, C7H16(417):0.00463, O2(2):0.10880, N2:0.88035'                      #need change if cti changes

gas.TPX = 791.7, 40*1e5, X_string
reactor = ct.IdealGasReactor(gas)
reactor_network = ct.ReactorNet([reactor])
import re
b = re.findall("\d+: \d+\.\d+", e)
for string in b:
    component_index = int(string.split()[0][:-1])
    print(reactor_network.component_name(component_index))

IdealGasReactor_3: S(30684)
IdealGasReactor_3: H(3)
IdealGasReactor_3: S(31941)
IdealGasReactor_3: HO2(10)
IdealGasReactor_3: S(31942)
IdealGasReactor_3: S(31940)
IdealGasReactor_3: C7H15(72)
